In [1]:
from gen_tables import setup_aws_credentials

session, creds = setup_aws_credentials()

In [2]:
import duckdb
import os

aws_acct_id = os.getenv("aws_acct_nbr")

cn = duckdb.connect()
cn.execute("install iceberg; load iceberg;")

cn.execute("""
    create or replace secret aws_sec (
           type s3,
           provider credential_chain
           )
""")

cn.execute(f"""
   attach '{aws_acct_id}' as iceberg_catalog (
       type iceberg,
       endpoint_type glue
    )
""")

cn.sql("""
    select *
    from iceberg_catalog.icebox1.iceberg_test
    limit 10   
""")


┌────────┬──────────────────────────────────────┬────────────┬──────────┐
│ row_id │               txn_key                │   rpt_dt   │ some_val │
│ int64  │               varchar                │    date    │  double  │
├────────┼──────────────────────────────────────┼────────────┼──────────┤
│      1 │ 0a83d754-9420-4ceb-9eec-3f7ab2495b9f │ 2025-09-21 │    20.05 │
│      2 │ aac7a8cf-2ee3-4e88-9fbe-ceaec99e0897 │ 2025-09-21 │    54.37 │
│      3 │ 417d33f7-5d43-41ea-b397-d28a88ebf2fd │ 2025-09-21 │    19.81 │
│      4 │ b06d7b8e-e3da-4030-a639-5a6830598f40 │ 2025-09-21 │    82.95 │
│      5 │ 324e646b-9eaf-4406-b414-73fef3e12ad6 │ 2025-09-21 │     2.37 │
│      6 │ 44358344-cee3-42b9-a173-76d9059a256e │ 2025-09-21 │    69.64 │
│      7 │ 59da140e-74a7-4ace-bf6b-92d3308f4201 │ 2025-09-21 │    32.54 │
│      8 │ d7a09520-7ed6-4af8-ae23-408f788c2dc1 │ 2025-09-21 │    70.97 │
│      9 │ 853921e7-4f09-4128-9d51-e08b58b74b0a │ 2025-09-21 │    47.57 │
│     10 │ 8fed505d-2880-4ec6-857e-207

In [ ]:
cn.execute(f"""
   attach or replace 'arn:aws:s3tables:us-east-1:{aws_acct_id}:bucket/icehouse-tbl-bucket1' as s3cat (
       type iceberg,
       endpoint_type s3_tables
    )
""")

In [9]:
cn.sql("""
    select *
       from s3cat.ns1.daily_sales
       limit 10
""")

┌────────────┬──────────────────┬──────────────┐
│ sale_date  │ product_category │ sales_amount │
│    date    │     varchar      │    double    │
├────────────┼──────────────────┼──────────────┤
│ 2024-01-15 │ Laptop           │        900.0 │
│ 2024-01-15 │ Monitor          │        250.0 │
│ 2024-01-16 │ Laptop           │       1350.0 │
│ 2024-02-01 │ Monitor          │        300.0 │
│ 2024-02-01 │ Keyboard         │         60.0 │
│ 2024-02-02 │ Mouse            │         25.0 │
│ 2024-02-02 │ Laptop           │       1050.0 │
│ 2024-02-03 │ Laptop           │       1200.0 │
│ 2024-02-03 │ Monitor          │        375.0 │
└────────────┴──────────────────┴──────────────┘

In [14]:
cn.sql("""
    create table s3cat.ns1.test_v2 as
       select cast(1 as int) as x, cast(2 as int) as y, cast('s' as varchar) as z
       
""") ## issues with the data types...go figure

In [15]:
cn.sql("""
    insert into s3cat.ns1.test_v2 values (3,4,'t')   
""")

In [16]:
cn.sql("drop table s3cat.ns1.test_v2")

In [17]:
cn.sql("drop table s3cat.ns1.daily_sales")

In [20]:
cn.sql('drop namespace s3cat.ns1')

ParserException: Parser Error: syntax error at or near "namespace"

LINE 1: drop namespace s3cat.ns1
             ^